In [1]:
import numpy  as np
import matplotlib.pyplot as plt

# 建模率评价

In [2]:
from draw import get_value_center_poisson,draw_section,draw_multi_tsne_distribution,data_root,get_value_center

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def get_modelable(meshs):
    j = 0
    meshs = meshs.reshape(-1,18,68,3)
    num = meshs.shape[0]
    for i in range(num):
        mesh = meshs[i].reshape(-1,3)
        
        is_inter,is_water,v,c = get_value_center(mesh) 
        if  is_inter or  not is_water or v==-1:
            j+=1
            # print(i)

    return (num - j)/num

In [4]:
dr = data_root()+ 'mesh_data_test_modified.npy'
data_list = ['complicated_ffd.npy','complicated_ffdGAN.npy','complicated_ffdpca.npy','complicated_ffdpcaGAN.npy']
data_list = [dr]+['data_set/'+_ for _ in data_list]

In [5]:
test = np.load(data_list[0])
a = get_modelable(test)


[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh


In [5]:
for i in range(1,len(data_list)):
    tmp = np.load(data_list[i])
    a = get_modelable(tmp)
    print(a)

0.3671875
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in

In [8]:
print(1-27/3634)

0.992570170610897


# VOD评价
relative variance of difference

$VOD(x) = \frac{1}{(M-1)*(N-1)} \sum_{i=0}^{M-1} \sum_{j=0}^{N-1} Var( \boldsymbol{x_{i+1,j} } -  \boldsymbol{x_{i,j }} )  + Var( \boldsymbol{x_{i,j+1} } -  \boldsymbol{x_{i,j }})$

$\operatorname{RVOD}=\frac{\mathbb{E}_{\boldsymbol{x} \sim P_{d a t a}} \operatorname{VOD}(\boldsymbol{x})}{\mathbb{E}_{\boldsymbol{x} \sim P_G} \operatorname{VOD}(\boldsymbol{x})} $


In [32]:
def get_vod(meshs):

    j = 0
    meshs = meshs.reshape(-1,18,68,3)
    num = meshs.shape[0]
        
    meshs01 = meshs[:,:-1,:,:].reshape(-1,3)
    meshs1  = meshs[:,1:,:,:].reshape(-1,3)
    m1      = meshs01 - meshs1
    meshs02 = meshs[:,:,:-1,:].reshape(-1,3)
    meshs2  = meshs[:,:,1:,:].reshape(-1,3)
    m2      = meshs02 - meshs2
    
    v1 = np.mean(np.var(m1,-1))
    v2 = np.mean(np.var(m2,-1))
    # v1 = np.mean(np.mean(np.power(m1 - np.mean(m1,-1)[:,None],2),-1))
    # v2 = np.mean(np.mean(np.power(m2 - np.mean(m2,-1)[:,None],2),-1))

    return v1 + v2

def get_vod2(meshs):

    j = 0
    meshs = meshs.reshape(-1,18,68,3)
    num = meshs.shape[0]
        
    meshs01 = meshs[:,:-2,:,:].reshape(-1,3)
    meshs1  = meshs[:,1:-1,:,:].reshape(-1,3)
    meshs11 = meshs[:,2:,:,:].reshape(-1,3)
    m1      = meshs01 - 2*meshs1 + meshs11

    meshs02 = meshs[:,:,:-2,:].reshape(-1,3)
    meshs2  = meshs[:,:,1:-1,:].reshape(-1,3)
    meshs12 = meshs[:,:,2:,:].reshape(-1,3)
    m2      = meshs02 - 2*meshs2 + meshs12
    
    v1 = np.mean(np.var(m1,-1))
    v2 = np.mean(np.var(m2,-1))
    # v1 = np.mean(np.mean(np.power(m1 - np.mean(m1,-1)[:,None],2),-1))
    # v2 = np.mean(np.mean(np.power(m2 - np.mean(m2,-1)[:,None],2),-1))

    return v1 + v2

In [25]:
test = np.load(data_list[0])
vod_real = get_vod(test)
print(vod_real)

0.000565560711336999


In [27]:
dr = data_root()+ 'mesh_data_test_modified.npy'
data_list = ['complicated_ffd_vari.npy','complicated_ffdGAN.npy','complicated_ffdpca_vari.npy','complicated_ffdpcaGAN.npy']
data_list = [dr]+['data_set/'+_ for _ in data_list]

In [30]:
for i in range(1,len(data_list)):
    tmp = np.load(data_list[i])
    a = get_vod(tmp)
    print(a,1- vod_real/a)

0.0006084244231669636 0.07045034715544607
0.0005669025000665407 0.0023668774249262503
0.000625179854300647 0.09536318637512808
0.0005710152622985834 0.00955237332821457


In [33]:
test = np.load(data_list[0])
vod_real2 = get_vod2(test)
print(vod_real2)

6.830665958326417e-06


In [37]:
for i in range(1,len(data_list)):
    tmp = np.load(data_list[i])
    a = get_vod2(tmp)
    print( vod_real2/a)

0.4507459377158516
1.3495429542671944
0.3497063993202588
0.3110043833107743


In [39]:
(0.902-0.7898)/(1-0.70898)

0.38554051267954126